In [2]:
import pandas as pd

In [4]:
df_1_1 = pd.read_excel('SUP_old.xlsx')
df_1_2 = pd.read_excel('TMC_old.xlsx')

c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
df1 = pd.concat([df_1_1,df_1_2],ignore_index=True)

In [5]:
df_2_1 = pd.read_excel('SUP_new.xlsx')
df_2_2 = pd.read_excel('TMC_new.xlsx')

c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
df2 = pd.concat([df_2_1,df_2_2],ignore_index=True)

In [8]:


def find_status_changes(df1, df2, key_column='Код КССС', status_column='Статус записи', comment_column='Комментарий'):
    """
    Сравнивает два датафрейма по ключу и находит записи с измененным статусом
    
    Parameters:
    df1, df2: DataFrame - датафреймы для сравнения
    key_column: str - название ключевой колонки
    status_column: str - название колонки со статусом
    comment_column: str - название колонки с комментарием
    
    Returns:
    DataFrame - записи с измененным статусом
    """
    
    # Объединяем датафреймы по ключу
    merged = pd.merge(
        df1[[key_column, status_column, comment_column]], 
        df2[[key_column, status_column, comment_column]], 
        on=key_column, 
        suffixes=('_old', '_new')
    )
    
    # Находим записи с измененным статусом
    status_changes = merged[merged[f'{status_column}_old'] != merged[f'{status_column}_new']]
    
    # Формируем итоговый датафрейм
    result = status_changes[[key_column]].copy()
    
    # Добавляем старый и новый статус
    result[f'{status_column}_старый'] = status_changes[f'{status_column}_old']
    result[f'{status_column}_новый'] = status_changes[f'{status_column}_new']
    
    # Добавляем комментарии (берем из нового датафрейма)
    result[f'{comment_column}'] = status_changes[f'{comment_column}_new']
    
    # Добавляем остальные поля из первого датафрейма
    other_columns = [col for col in df1.columns if col not in [key_column, status_column, comment_column]]
    if other_columns:
        other_data = df1[other_columns + [key_column]].merge(result[[key_column]], on=key_column)
        result = result.merge(other_data, on=key_column)
    
    return result.reset_index(drop=True)


    


In [9]:

    
print("Датафрейм 1:")
print(df1)
print("\nДатафрейм 2:")
print(df2)
    
    # Находим изменения
changes = find_status_changes(df1, df2)
    
print("\nЗаписи с измененным статусом:")
print(changes)


Датафрейм 1:
         Код класса ТМЦ и услуг  \
0                  2.002221e+15   
1                  2.002221e+15   
2                  2.002221e+15   
3                  2.002221e+15   
4                  2.002221e+15   
...                         ...   
1283331            7.010907e+14   
1283332            1.711010e+15   
1283333            1.711010e+15   
1283334            1.711010e+15   
1283335            1.711040e+15   

                         Наименование класса ТМЦ и услуг  Код КССС  \
0                    Уплотнения, прокладки и уплотнители    948278   
1                    Уплотнения, прокладки и уплотнители   1590013   
2                    Уплотнения, прокладки и уплотнители    948282   
3                    Уплотнения, прокладки и уплотнители    948283   
4                    Уплотнения, прокладки и уплотнители   3117191   
...                                                  ...       ...   
1283331                             Хозяйственные товары   2113286   
128333

In [10]:
changes.to_excel('block.xlsx')

In [11]:
changes.head()

,Код КССС,Статус записи_старый,Статус записи_новый,Комментарий,Код класса ТМЦ и услуг,Наименование класса ТМЦ и услуг,Краткое наименование,Полное наименование,Базовая единица измерения,Альтернативные единицы измерения,Продукция группы ЛУКОЙЛ,Тип записи,Материал верхнего уровня
0,3365891,1,0,Переведено в статус На блокирование в рамках п...,2.002221e+15,"Уплотнения, прокладки и уплотнители",Кольцо упл. VG 195000-214 сист. KFDJ-500,Кольцо уплотнительное Vetco Gray 195000-214 си...,108,89,False,3,NaN
1,1403092,1,0,Переведено в статус На блокирование в рамках п...,2.002221e+15,"Уплотнения, прокладки и уплотнители","Прокладка ПАГФ-Г-П1-2,5-03-309х286х3","Прокладка фланцевая ПАГФ-Г-П1-2,5-03-309х286х3...",108,NaN,False,3,NaN
2,1057849,1,0,Временная разблокировка,2.002221e+15,"Уплотнения, прокладки и уплотнители",Техпластина силиконовая 4х500х1500,Техпластина силиконовая 4х500х1500 ТУ 2500-281...,117,122,False,3,NaN
3,1057851,1,0,Временная разблокировка,2.002221e+15,"Уплотнения, прокладки и уплотнители",Техпластина силиконовая 3х500х1500,Техпластина силиконовая 3х500х1500 ТУ 2500-281...,117,122,False,3,NaN
4,1403096,1,0,Переведено в статус На блокирование в рамках п...,2.002221e+15,"Уплотнения, прокладки и уплотнители","Прокладка ПАГФ-Г-П1-2,5-03-900х800х3","Прокладка фланцевая ПАГФ-Г-П1-2,5-03-900х800х3...",108,NaN,False,3,NaN


In [12]:
df2[df2.duplicated(subset=['Краткое наименование','Полное наименование'], keep=False)]

,Код класса ТМЦ и услуг,Наименование класса ТМЦ и услуг,Код КССС,Краткое наименование,Полное наименование,Базовая единица измерения,Альтернативные единицы измерения,Продукция группы ЛУКОЙЛ,Статус записи,Тип записи,Материал верхнего уровня,Комментарий,Дата создания в КССС
12,2.017010e+15,Узлы машин для газопламенной обработки,2118010,Наконечник для РЗП-02,Наконечник для резака РЗП-02,108,NaN,False,0,4,NaN,s4,15.06.2024
25,2.017010e+15,Узлы машин для газопламенной обработки,2144004,Рампа разрядная азотная,Рампа разрядная азотная,108,NaN,False,0,4,NaN,согласовать как номенклатурная позиция на осно...,20.08.2024
90,2.017010e+15,Узлы машин для газопламенной обработки,2113428,Редуктор кислородный БКО-50-2,Редуктор баллонный кислородный одноступенчатый...,108,89,False,0,4,NaN,NaN,04.06.2024
91,2.017010e+15,Узлы машин для газопламенной обработки,2217393,Клапан вентиля кислородного ВК-94,Клапан вентиля кислородного ВК-94,108,NaN,False,0,4,NaN,КП пункт 15,09.01.2025
195,2.002130e+15,Теплообменники кожухотрубчатые,1470839,Теплообменник Т-118,Теплообменник кожухотрубчатый Т-118,108,89,False,1,3,NaN,Переведено в статус На блокирование в рамках п...,05.06.2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373332,2.001021e+15,Части противовыбросового оборудования прочие,3401741,Уплотнение торц. Cameron 140398-04-01-03,"Уплотнение торцевое в сборе 3,376x3,645 Camero...",108,NaN,False,0,3,NaN,NaN,03.04.2024
1373333,2.001021e+15,Части противовыбросового оборудования прочие,3401742,Уплотнение торц. Cameron 140399-04-01-03,"Уплотнение торцевое в сборе 4,022 x4,294 Camer...",108,NaN,False,0,3,NaN,NaN,03.04.2024
1373334,2.001021e+15,Части противовыбросового оборудования прочие,3401743,Маховик в сборе Cameron 2047076-02,"Маховик в сборе 18,50 без рукояток Cameron 204...",108,NaN,False,0,3,NaN,NaN,03.04.2024
1373484,7.011105e+14,Канцелярские товары,2268721,Папка-портфель,Папка-портфель,108,NaN,False,0,4,NaN,NaN,16.04.2025


In [13]:
doubles = df2[df2.duplicated(subset=['Краткое наименование','Полное наименование'], keep=False)]

In [14]:
df_sorted = doubles.sort_values(by = 'Краткое наименование')

In [15]:
df_sorted.head()

,Код класса ТМЦ и услуг,Наименование класса ТМЦ и услуг,Код КССС,Краткое наименование,Полное наименование,Базовая единица измерения,Альтернативные единицы измерения,Продукция группы ЛУКОЙЛ,Статус записи,Тип записи,Материал верхнего уровня,Комментарий,Дата создания в КССС
1001291,6.020212e+14,Жидкости автохимические и рабочие фасов.,1181005,!!!G 7016 Автополироль для кузова 437 мл,!!!G 7016 Автополироль для кузова 437 мл,108,NaN,False,1,3,NaN,NaN,01.04.2024
837033,7.010200e+14,Автохимические жидкости (не нефтехимия),3487763,!!!G 7016 Автополироль для кузова 437 мл,!!!G 7016 Автополироль для кузова 437 мл,108,NaN,False,0,3,NaN,NaN,31.03.2024
837036,7.010200e+14,Автохимические жидкости (не нефтехимия),3487766,--G-10307 Средство для удаления мелких ц,G-10307 Средство для удаления мелких царапин с...,108,NaN,False,0,3,NaN,NaN,31.03.2024
1001303,6.020212e+14,Жидкости автохимические и рабочие фасов.,1181099,--G-10307 Средство для удаления мелких ц,G-10307 Средство для удаления мелких царапин с...,108,NaN,False,1,3,NaN,NaN,01.04.2024
808653,6.020212e+14,Жидкости автохимические и рабочие фасов.,3487767,--RD 50 Присадка к дизтопливу и очистите,RD 50 Присадка к дизтопливу и очиститель форсу...,108,NaN,False,0,3,NaN,NaN,31.03.2024


In [16]:
df_sorted.to_excel('дубли.xlsx')

In [30]:
df2[(df2['Дата создания в КССС'] > '01.01.2025') & (df2['Тип записи'] == 4) & (df2['Статус записи'] == 0)]

,Код класса ТМЦ и услуг,Наименование класса ТМЦ и услуг,Код КССС,Краткое наименование,Полное наименование,Базовая единица измерения,Альтернативные единицы измерения,Продукция группы ЛУКОЙЛ,Статус записи,Тип записи,Материал верхнего уровня,Комментарий,Дата создания в КССС
4,2.002221e+15,"Уплотнения, прокладки и уплотнители",3117191,Прокладка Y751-40281,Прокладка Y751-40281,108,NaN,False,0,4,NaN,NaN,05.06.2023
7,2.002221e+15,"Уплотнения, прокладки и уплотнители",3117192,Прокладка Y753-47171,Прокладка Y753-47171,108,NaN,False,0,4,NaN,NaN,05.06.2023
9,2.002221e+15,"Уплотнения, прокладки и уплотнители",3117200,"Прокладка рифленая SS304L, JT2","Прокладка рифленая SS304L, JT2",108,NaN,False,0,4,NaN,NaN,05.06.2023
11,2.002221e+15,"Уплотнения, прокладки и уплотнители",2138695,"Кольцо G1/2, М20х1,5, ПМБ, 18x6x3мм","Кольцо уплотнительное G1/2, М20х1,5, ПМБ, 18x6...",108,111,False,0,4,NaN,Кп пункт 19 и 40,07.08.2024
12,2.017010e+15,Узлы машин для газопламенной обработки,2118010,Наконечник для РЗП-02,Наконечник для резака РЗП-02,108,NaN,False,0,4,NaN,s4,15.06.2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373472,7.011105e+14,Канцелярские товары,3703076,"Ручка STAFF корректирующая College, 4мл","Ручка STAFF корректирующая College, 4мл, метал...",108,NaN,False,0,4,NaN,NaN,19.02.2024
1373473,7.011105e+14,Канцелярские товары,3703078,Лента Attache клейкая двустор. 38ммх10м,"Лента Attache клейкая двусторонняя 38мм х 10м,...",108,NaN,False,0,4,NaN,NaN,19.02.2024
1373478,7.010907e+14,Хозяйственные товары,3683308,"Материал расходный-грунт, субстраты","Материал расходный-грунт, субстраты, препараты...",108,NaN,False,0,4,NaN,NaN,22.12.2023
1373484,7.011105e+14,Канцелярские товары,2268721,Папка-портфель,Папка-портфель,108,NaN,False,0,4,NaN,NaN,16.04.2025


In [31]:
new_sup = df2[(df2['Дата создания в КССС'] > '01.01.2025') & (df2['Тип записи'] == 4) & (df2['Статус записи'] == 0)]

In [32]:
new_sup.to_excel('new_supl.xlsx')